In [1]:
import pandas as pd

In [2]:
test_path = "hdfs://namenode:9000/data/Weather Test Data.csv"
train_path = "hdfs://namenode:9000/data/Weather Training Data.csv"

### Initialize Spark

In [3]:
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession

In [4]:
sc = SparkContext(master="local[*]")

In [5]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [ ]:
"Location"
"MinTemp"
"MaxTemp"
"Rainfall"
"Evaporation"
"Sunshine"
"WindGustDir"
"WindGustSpeed"
"WindDir9am"
"WindDir3pm"
"WindSpeed9am"
"WindSpeed3pm"
"Humidity9am"
"Humidity3pm"
"Pressure9am"
"Pressure3pm"
"Cloud9am"
"Cloud3pm"
"Temp9am"
"Temp3pm"
"RainToday"

In [7]:
spark = SparkSession.builder.appName('australian-weather-prediction-spark').getOrCreate()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:34889)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:34889)

### Load Dataset

In [ ]:
df_train = spark.read.format("csv").option("header", "true").load(train_path)